Read cytobands(hg38) from a file, for each dataset, count the overlaps in each band than save to db.  
This is the first version using dataframe, it was too slow, the new verison uisng List is much faster.  
Depreciated version.

In [133]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

Read cytoband info from a file. Remove chromosome piecies, only keep 1-22,X,Y. Intit counting params

In [225]:

bands = pd.read_csv('/Users/bogao/DataFiles/Data/genome/cytoBand_hg38.txt',sep='\t', header=None,
                     names=['chro','start','end','name','note'])

bands = bands[~bands.chro.str.contains('_|M')]

bands = bands.assign(total_amp=0, total_dup=0, total_length=0, count=0, length=0)

The function version, details not updated to the final version

In [149]:
def cytobandOverlaps(bands, collection, label):
    db = MongoClient()['Rebased'][collection]
    samples = []
    for sam in db.find():
        samples.append(sam)

    c = 1
    for sam in samples:
        for seg in sam['segments']:
            chrom = ''.join(['chr', seg['chro']])
            # update length in each
            sizes = []
            for index, row in bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start'])].iterrows():
                size = min(row['end'],seg['end']) - max(row['start'],seg['start'])
                sizes.append(size)

            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'length'] = sizes

            # update count
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'count'] +=1

            # update total_length
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'total_length'] += bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'length']

            # update signal
            if seg['value'] > 0:
                bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'total_amp'] += seg['value']
            else:
                bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'total_dup'] += seg['value']
        print(c, end='\r')
        c +=1
    bands_dict = bands.to_dict('records')
    db = MongoClient()['Rebased'][label]
    db.drop()
    db.insert_many(bands_dict)

Convert to records and save to db

In [151]:
breast_bands = bands.to_dict('records')

db = MongoClient()['Rebased']['bands_breast_normalized']
db.drop()
db.insert_many(breast_bands)

Testings code 

In [165]:
s = 3301765
e = 13301765
# bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & 
#           (bands['end'] >= s),['segstart','segend']] = [s,e]


sizes = []
for index, row in bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)].iterrows():
    size = min(row['end'],e) - max(row['start'],s)
    sizes.append(size)



bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & 
          (bands['end'] >= s), 'length'] = sizes

bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & 
          (bands['end'] >= s), 'count'] +=1

bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'total_length'] += bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'length']

bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)]

sizes = []
for index, row in bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)].iterrows():
    size = min(row['end'],e) - max(row['start'],s)
    sizes.append(size)

pd.Series({'total_signal':[1,2,3,4,5], 'count':[6,7,8,9,0]})

bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < val) & (bands['end'] >= val)]

IndentationError: unexpected indent (<ipython-input-165-4dbda3d24333>, line 9)

This block was using the same algorithm as the later List version.

In [229]:
db = MongoClient()['Rebased']['skin']
bands = bands.assign(total_dup=0, total_del=0, dup_length=0, del_length=0, dup_count=0, del_count=0, length=0)
samples = []
for sam in db.find():
    samples.append(sam)

c = 1
for sam in samples:
    for seg in sam['segments']:
        chrom = ''.join(['chr', seg['chro']])
        # update length in each
        sizes = []
        values = []
        for index, row in bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start'])].iterrows():
            size = min(row['end'],seg['end']) - max(row['start'],seg['start'])
            sizes.append(size)
            values.append(size*seg['value'])

        bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'length'] = sizes



        
        # dup
        if seg['value'] > 0:
            # update signal
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'total_dup'] += values
            # update count
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'dup_count'] +=1
            # update total_length
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'dup_length'] += bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'length']
        
        # del
        else:
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'total_del'] += values
            # update count
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'del_count'] +=1
            # update total_length
            bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'del_length'] += bands.loc[ (bands['chro'] ==chrom) & (bands['start'] < seg['end']) & (bands['end'] >= seg['start']), 'length']

            
    print(c, end='\r')
    c +=1
    
#     if c>1:
#         break

Compute average and check

In [222]:
bands['ave_dup'] = bands['total_dup']/bands['dup_length']
bands['ave_del'] = bands['total_del']/bands['del_length']

bands

In [224]:
skin_bands = bands.to_dict('records')
db = MongoClient()['Rebased']['bands_skin_original']
db.drop()
db.insert_many(skin_bands)

The following blocks were to testing Dataframe speed, there were too slow.

In [177]:
def testloop():
    sizes = []
    for index, row in bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)].iterrows():
        size = min(row['end'],e) - max(row['start'],s)
        sizes.append(size)
    bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'length'] = sizes
    bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'count'] +=1
    bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'total_length'] += bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'length']
%timeit testloop()

100 loops, best of 3: 15.1 ms per loop


In [191]:
def testloop1():
    sizes = []
    dd = bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)]
    for index, row in dd.iterrows():
        size = min(row['end'],e) - max(row['start'],s)
        sizes.append(size)
    dd['length'] = sizes
    dd['count'] +=1
    dd['total_length'] += dd['length']
%timeit testloop()

100 loops, best of 3: 15.2 ms per loop


In [174]:
sizes = []
for index, row in bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)].iterrows():
    size = min(row['end'],e) - max(row['start'],s)
    sizes.append(size)

In [175]:
%timeit bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'length'] = sizes

100 loops, best of 3: 3.35 ms per loop


In [180]:
15.1*500*2000/1000/60

251.66666666666666

In [169]:
%timeit bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'count'] +=1

100 loops, best of 3: 4.01 ms per loop


In [170]:
%timeit bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'total_length'] += bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s), 'length']

100 loops, best of 3: 5.33 ms per loop


In [171]:
%timeit bands.loc[ (bands['chro'] =='chr1') & (bands['start'] < e) & (bands['end'] >= s)]

1000 loops, best of 3: 1.53 ms per loop


In [188]:
chr22 = []
db = db = MongoClient()['Rebased']['breast']
for sam in db.find():
    for seg in sam['segments']:
        if seg['chro'] == '22':
            chr22.append(seg)

In [189]:
df22 = pd.DataFrame(chr22)